In [31]:
import pandas as pd
data = pd.read_csv(r"C:\Users\honey\Desktop\file.csv")

In [32]:
data.head()


,cigg_per_day,haemoglobin_concentration_per_deciletre,years_of_smoking,heart_beat_deviation,percentage_of_choked_pipe,percentage_of_lungs_infected,case
0,6,17.60,2,18,36,39,2
1,2,16.95,1,6,15,29,1
2,3,17.45,4,18,31,43,2
3,1,16.80,1,3,10,21,1
4,5,18.12,4,20,39,49,3


In [33]:
label=data.pop('case')

In [34]:
label.head()

0    2
1    1
2    2
3    1
4    3
Name: case, dtype: int64

In [35]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [36]:
col_for_norm=['cigg_per_day','haemoglobin_concentration_per_deciletre','years_of_smoking','heart_beat_deviation','percentage_of_choked_pipe','percentage_of_lungs_infected']
data[col_for_norm]=data[col_for_norm].apply(lambda x:(x-x.min())/(x.max()-x.min()))

In [37]:
data.head()

,cigg_per_day,haemoglobin_concentration_per_deciletre,years_of_smoking,heart_beat_deviation,percentage_of_choked_pipe,percentage_of_lungs_infected
0,0.714286,0.533333,0.333333,0.75,0.787879,0.642857
1,0.142857,0.100000,0.000000,0.15,0.151515,0.285714
2,0.285714,0.433333,1.000000,0.75,0.636364,0.785714
3,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,0.571429,0.880000,1.000000,0.85,0.878788,1.000000


In [38]:
train_x,test_x,train_y,test_y=train_test_split(data,label)

In [39]:
train_x.shape

(5, 6)

In [41]:
cigg_per_day=tf.feature_column.numeric_column('cigg_per_day')
haemoglobin_concentration=tf.feature_column.numeric_column('haemoglobin_concentration_per_deciletre')
years_of_smoking=tf.feature_column.numeric_column('years_of_smoking')
heart_beat_deviation=tf.feature_column.numeric_column('heart_beat_deviation')
percentage_of_choked_pipe=tf.feature_column.numeric_column('percentage_of_choked_pipe')
percentage_of_lungs_infected=tf.feature_column.numeric_column('percentage_of_lungs_infected')

In [42]:
feat_cols=[cigg_per_day,haemoglobin_concentration,years_of_smoking,heart_beat_deviation,percentage_of_choked_pipe,percentage_of_lungs_infected]

In [48]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feat_cols,
    hidden_units=[30, 10],
    n_classes=4)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\honey\\AppData\\Local\\Temp\\tmpq6eovlvi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [49]:
input_fns=tf.compat.v1.estimator.inputs.pandas_input_fn(x=train_x,y=train_y,batch_size=10,num_epochs=100,shuffle=True)

In [50]:
classifier.train(
    input_fns,
    steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\honey\AppData\Local\Temp\tmpq6eovlvi\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


InvalidArgumentError: Graph execution error:

assertion failed: [Labels must be <= n_classes - 1] [Condition x <= y did not hold element-wise:] [x (head/losses/labels:0) = ] [[2][2][1]...] [y (head/losses/check_label_range/Const:0) = ] [3]
	 [[{{node Assert}}]]

In [51]:
input_fnt=tf.compat.v1.estimator.inputs.pandas_input_fn(x=test_x,batch_size=10,num_epochs=100,shuffle=False)

In [52]:
res=classifier.evaluate(input_fnt)
print(res)

INFO:tensorflow:Calling model_fn.


ValueError: You must provide a labels Tensor. Given: None. Suggested troubleshooting steps: Check that your data contains your label feature. Check that your input_fn properly parses and returns labels.